## Calcolo del punteggio ottenuto

In [6]:
import pandas as pd

# Carica i file
pred = pd.read_csv('./dataset/outputs/final_test_data_predictions.csv')
gt = pd.read_csv('./dataset/test/answer_competition.csv')

# Rinominare 'Case' in 'ID' per il merge
pred = pred.rename(columns={"Case": "ID"})

# Aggiungi colonna 'Spacecraft No.' a pred prendendola dal ground truth
pred = pred.merge(gt[['ID', 'Spacecraft No.']], on='ID', how='left')

# Merge completo per confronto con tutti i task
merged = pred.merge(gt, on=['ID', 'Spacecraft No.'], suffixes=('_pred', '_gt'))

# Inizializza punteggio totale e punteggio massimo possibile
total_score = 0
max_possible_score = 0

# Ciclo su ogni riga
for _, row in merged.iterrows():
    multiplier = 2 if row['Spacecraft No.'] == 4 else 1
    score = 0
    max_score = 10  # Task 1 è sempre valutabile

    # --- TASK 1: Normal/Abnormal ---
    pred_abnormal = 1 if row['Condition'] in [2, 3] else 0
    gt_abnormal = row['task1']
    if pred_abnormal == gt_abnormal:
        score += 10

        # --- TASK 2: Tipo di anomalia (solo se anomalia corretta) ---
        max_score += 10
        if pred_abnormal == 1 and row['Condition'] == row['task2']:
            score += 10

            # --- TASK 3 ---
            if row['Condition'] == 2:
                max_score += 10
                if row['anomaly_location'] == row['task3']:
                    score += 10

            # --- TASK 4 ---
            if row['Condition'] == 3:
                max_score += 10
                if row['fault_location'] == row['task3']:
                    score += 10

                # --- TASK 5: apertura valvola ---
                max_score += 20
                diff = abs(row['valve_opening'] - row['task4'])
                valve_score = max(-diff + 20, 0)
                score += valve_score

    total_score += score * multiplier
    max_possible_score += max_score * multiplier

# Percentuale
percentage = (total_score / max_possible_score) * 100 if max_possible_score > 0 else 0

print(f"🎯 Punteggio totale: {total_score}")
print(f"📊 Percentuale: {percentage:.2f}%")


🎯 Punteggio totale: 890
📊 Percentuale: 53.94%
